In [2]:
import pandas as pd
import re
import os

#### Filter the Plink variants for just SNPs

In [ ]:
variants_plink="/gchm/cd4_QTL_analysis/01_genotype_snps_covar/02_genotype_covariates/analysis/002.v4_calculating_pcs_MAF5/CD4_all_chr_ashkenazi.364.AF1.QC.BA.king2.hwe.annot"
vars_out="input/CD4_all_chr_ashkenazi.364.AF1.QC.BA.king2.hwe.annot.snps"

!plink2 --bfile $variants_plink --snps-only just-acgt --make-bed --out ${vars_out}

/bin/bash: line 1: plink2: command not found


#### Read the SNPs

In [13]:
df = pd.read_table("input/CD4_all_chr_ashkenazi.364.AF1.QC.BA.king2.hwe.annot.snps.bim",
                         sep='\t', header=None)
df.columns = ['chr', 'variant_id', 'genetic_dist', 'pos', 'allele1', 'allele2']

df.head

<bound method NDFrame.head of          chr           variant_id  genetic_dist        pos allele1 allele2
0         10     10:61172[b38]A,G             0      61172       G       A
1         10     10:65673[b38]C,T             0      65673       T       C
2         10     10:68110[b38]G,C             0      68110       C       G
3         10     10:70493[b38]C,A             0      70493       A       C
4         10     10:76277[b38]G,T             0      76277       T       G
...      ...                  ...           ...        ...     ...     ...
5810462    9  9:138201870[b38]C,A             0  138201870       A       C
5810463    9  9:138202443[b38]C,T             0  138202443       T       C
5810464    9  9:138202814[b38]C,T             0  138202814       T       C
5810465    9  9:138206150[b38]G,A             0  138206150       A       G
5810466    9  9:138208331[b38]T,G             0  138208331       G       T

[5810467 rows x 6 columns]>

#### Edit the variant_ids and also set allele1/2 as Ref/Atl, not major/minor like Plink format

In [14]:
# Extract alleles from variant_id
variants = df.copy()
variants[['allele1', 'allele2']] = variants['variant_id'].str.extract(r'.*\](.*),(.*)\s*$')

# Create new variant_id format with 'chr' prefix
variants['variant_id'] = variants['variant_id'].apply(
    lambda x: re.sub(r'(\d+):(\d+)\[b38\](.*),(.*)\s*', r'chr\1_\2_\3_\4', x) if pd.notna(x) else x
)

# Select and reorder columns
variants = variants[['chr', 'pos', 'allele1', 'allele2', 'variant_id']]
variants['chr'] = 'chr' + variants['chr'].astype(str)
variants

,chr,pos,allele1,allele2,variant_id
0,chr10,61172,A,G,chr10_61172_A_G
1,chr10,65673,C,T,chr10_65673_C_T
2,chr10,68110,G,C,chr10_68110_G_C
3,chr10,70493,C,A,chr10_70493_C_A
4,chr10,76277,G,T,chr10_76277_G_T
...,...,...,...,...,...
5810462,chr9,138201870,C,A,chr9_138201870_C_A
5810463,chr9,138202443,C,T,chr9_138202443_C_T
5810464,chr9,138202814,C,T,chr9_138202814_C_T
5810465,chr9,138206150,G,A,chr9_138206150_G_A


#### Read the peaks from chrombpnet and filter variants

In [11]:
peaks=pd.read_table("~/cd4_chrombpnet/data/inputs/peaks/merged_cd4_samples_peaks_no_blacklist.sorted.bed",
                         sep='\t', header=None)
peaks

,0,1,2,3,4,5,6,7,8,9
0,chr1,10003,10604,merged_cd4_samples_peak_1,3154,.,9.48084,315.49900,313.14900,110
1,chr1,10996,11452,merged_cd4_samples_peak_2,399,.,3.08842,39.93770,38.00440,282
2,chr1,15486,15745,merged_cd4_samples_peak_3,176,.,2.45185,17.60840,15.79440,186
3,chr1,17140,17900,merged_cd4_samples_peak_4,300,.,3.00455,30.02770,28.13810,334
4,chr1,20304,21778,merged_cd4_samples_peak_5a,31,.,1.52875,3.19981,1.61032,70
...,...,...,...,...,...,...,...,...,...,...
125949,chr22,50702244,50702805,merged_cd4_samples_peak_61127,249,.,3.04065,24.96390,23.10080,186
125950,chr22,50731899,50732864,merged_cd4_samples_peak_61128,4998,.,11.15230,499.81700,497.33900,439
125951,chr22,50756397,50757597,merged_cd4_samples_peak_61129,33759,.,20.33370,3375.95000,3372.14000,648
125952,chr22,50775188,50775523,merged_cd4_samples_peak_61130,76,.,1.96748,7.67437,5.96979,171


In [34]:
import pyranges as pr


def filter_variants_within_peaks(variants_df, peaks_df):
    # Create temporary copies with renamed columns for PyRanges
    variants_pr_df = variants_df.copy()
    peaks_pr_df = peaks_df.copy()
    
    # Rename for PyRanges (but don't modify the original)
    variants_pr_df = variants_pr_df.rename(columns={'chr': 'Chromosome', 'pos': 'Start'})
    variants_pr_df['End'] = variants_pr_df['Start'] - 1 + 1  # Convert position to 0-indexed and then add 1 for end
    
    peaks_pr_df = peaks_pr_df.rename(columns={
        '0': 'Chromosome',
        '1': 'Start',
        '2': 'End'
    })
    
    # Create PyRanges objects
    variants_pr = pr.PyRanges(variants_pr_df)
    peaks_pr = pr.PyRanges(peaks_pr_df)
    
    # Find overlaps
    overlaps = variants_pr.join(peaks_pr)
    
    # Extract just the variant_ids
    variant_ids_in_peaks = set(overlaps.df['variant_id'])
    
    # Filter the ORIGINAL variants dataframe to maintain all original columns
    filtered_variants = variants_df[variants_df['variant_id'].isin(variant_ids_in_peaks)]
    
    return filtered_variants

filtered_variants = find_variants_in_peaks(variants, peaks)


In [38]:
filtered_variants

,Chromosome,Start,allele1,allele2,variant_id,End,Start_b,End_b,3,4,5,6,7,8,9,original_pos
0,chr1,17558,G,C,chr1_17559_G_C,17559,17140,17900,merged_cd4_samples_peak_4,300,.,3.00455,30.02770,28.13810,334,17559
1,chr1,183802,G,A,chr1_183803_G_A,183803,183260,184667,merged_cd4_samples_peak_17a,35,.,1.42319,3.51459,1.91425,93,183803
2,chr1,183802,G,A,chr1_183803_G_A,183803,183260,184667,merged_cd4_samples_peak_17b,219,.,2.33050,21.93830,20.09420,573,183803
3,chr1,183802,G,A,chr1_183803_G_A,183803,183260,184667,merged_cd4_samples_peak_17c,193,.,2.21077,19.39190,17.56460,969,183803
4,chr1,183802,G,A,chr1_183803_G_A,183803,183260,184667,merged_cd4_samples_peak_17d,170,.,2.11368,17.01890,15.21020,1133,183803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242039,chr22,50674805,G,C,chr22_50674806_G_C,50674806,50674661,50675786,merged_cd4_samples_peak_61125,795,.,3.79752,79.56260,77.51180,430,50674806
242040,chr22,50775330,G,C,chr22_50775331_G_C,50775331,50775188,50775523,merged_cd4_samples_peak_61130,76,.,1.96748,7.67437,5.96979,171,50775331
242041,chr22,50775397,G,A,chr22_50775398_G_A,50775398,50775188,50775523,merged_cd4_samples_peak_61130,76,.,1.96748,7.67437,5.96979,171,50775398
242042,chr22,50783302,T,C,chr22_50783303_T_C,50783303,50783044,50784296,merged_cd4_samples_peak_61131,30880,.,14.51720,3088.06000,3084.42000,657,50783303


In [30]:
filtered_variants.shape
filtered_variants['variant_id'].nunique()
filtered_variants[3].nunique()

def calculate_mean_variants_per_peak(filtered_variants_df):
    # Count variants per peak
    variants_per_peak = filtered_variants_df.groupby(3).size()
    
    # Calculate mean
    mean_variants = variants_per_peak.mean()
    
    # Calculate additional statistics if needed
    median_variants = variants_per_peak.median()
    max_variants = variants_per_peak.max()
    min_variants = variants_per_peak.min()
    
    return {
        'mean': mean_variants,
        'median': median_variants,
        'max': max_variants,
        'min': min_variants,
        'total_peaks_with_variants': len(variants_per_peak)
    }
# Calculate mean variants per peak
stats = calculate_mean_variants_per_peak(filtered_variants)

print(f"Mean variants per peak: {stats['mean']:.2f}")
print(f"Median variants per peak: {stats['median']:.2f}")
print(f"Maximum variants in a peak: {stats['max']}")
print(f"Minimum variants in a peak: {stats['min']}")
print(f"Total peaks containing variants: {stats['total_peaks_with_variants']}")

Mean variants per peak: 2.91
Median variants per peak: 2.00
Maximum variants in a peak: 50
Minimum variants in a peak: 1
Total peaks containing variants: 83049


In [37]:
variant_ids_in_peaks = set(filtered_variants['variant_id'])
# Subset the original variants table using these IDs
variants_in_peaks = variants[variants['variant_id'].isin(variant_ids_in_peaks)]
print(variants_in_peaks.shape)
variants_in_peaks.head

(160248, 5)


<bound method NDFrame.head of            chr        pos allele1 allele2          variant_id
4        chr10      76277       G       T     chr10_76277_G_T
9        chr10      87333       G       C     chr10_87333_G_C
10       chr10      87401       C       T     chr10_87401_C_T
11       chr10      87928       G       A     chr10_87928_G_A
12       chr10      87983       T       C     chr10_87983_T_C
...        ...        ...     ...     ...                 ...
5810266   chr9  138022809       G       T  chr9_138022809_G_T
5810267   chr9  138022939       G       C  chr9_138022939_G_C
5810392   chr9  138127843       A       G  chr9_138127843_A_G
5810393   chr9  138128413       C       A  chr9_138128413_C_A
5810427   chr9  138147679       G       C  chr9_138147679_G_C

[160248 rows x 5 columns]>

In [39]:
# Write to TSV file
variants_in_peaks.to_csv('input/converted_variants_inpeaks.tsv', sep='\t', index=False, header=None)
